### PUNTO 2.a, esto después lo cambiamos

In [3]:
pip install scapy

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 33.4 MB/s eta 0:00:00


In [4]:
from scapy.all import *

#Agrego
from scapy.layers.inet import IP, ICMP
import time
##

def ping(host):
    print (f"Ping {host}")

    #1. Creamos el paquete ICMP
    pkt = IP(dst=host)/ICMP()

    # Medimos el tiempo de envío
    start_time = time.time()

    # Enviamos y recibimos el paquete (timeout de 2 segundos)
    resp = sr1(pkt, timeout=2, verbose=0)
    end_time = time.time()

    if resp is not None and ICMP in resp and resp[ICMP].type == 0:
        # RTT (Round Trip Time)
        rtt = (end_time - start_time) * 1000  # en milisegundos

        # Longitud del paquete
        length = len(resp)

        # TTL (Time to Live)
        ttl = resp.ttl

        print(f"Respuesta de {host}: longitud = {length} bytes | RTT = {rtt:.2f} ms | TTL = {ttl}")
    else:
        print(f"No se recibió respuesta de {host}")

##

ping("google.com")

Ping google.com
Respuesta de google.com: longitud = 28 bytes | RTT = 81.43 ms | TTL = 114


### PUNTO 2b

In [7]:
from scapy.all import *

#Agrego
from scapy.layers.inet import IP, ICMP
import socket ###hay que usar sockets?? lo agrego para hacer bien la resolucion de nombres

import time
import statistics

def ping_estadisticas(host, cantidad):
    print(f"Enviando {cantidad} paquetes ICMP a {host}...\n")

    enviados = 0
    recibidos = 0
    rtts = []

    for i in range(cantidad): ## agrego el fro para asegurarme para asegurarme que el nombre del host tenga ip

        try:
             ip_destino = socket.gethostbyname(host)
        except socket.gaierror:
            print(f"No se pudo resolver el nombre {host}. Abortando...") ##esto es error de DNs
            return

        pkt = IP(dst=ip_destino)/ICMP(id=0x1234, seq=i)

       ## pkt = IP(dst=host)/ICMP(id=0x1234, seq=i)
        enviados += 1

        start_time = time.time()
        resp = sr1(pkt, timeout=2, verbose=0)
        end_time = time.time()

        if resp is not None and ICMP in resp and resp[ICMP].type == 0:
            rtt = (end_time - start_time) * 1000  # RTT en ms
            rtts.append(rtt)
            recibidos += 1
            length = len(resp)
            ttl = resp.ttl
            print(f"[#{i+1}] Respuesta de {resp.src} | Longitud: {length} bytes | RTT: {rtt:.2f} ms | TTL: {ttl}")
        else:
            print(f"[#{i+1}] No se recibió respuesta válida")

    # Estadísticas
    perdidos = enviados - recibidos
    porcentaje_perdidos = (perdidos / enviados) * 100 if enviados > 0 else 0

    print("\nEstadísticas finales:")
    print(f"Paquetes enviados: {enviados}")
    print(f"Paquetes recibidos: {recibidos}")
    print(f"Paquetes perdidos: {perdidos}")
    print(f"Porcentaje de pérdida: {porcentaje_perdidos:.2f}%")

    if rtts:
        print(f"RTT promedio: {statistics.mean(rtts):.2f} ms")
        print(f" RTT máximo: {max(rtts):.2f} ms")
        print(f" RTT mínimo: {min(rtts):.2f} ms")
        if len(rtts) > 1:
            print(f"Desvío estándar del RTT: {statistics.stdev(rtts):.2f} ms")
        else:
            print("Desvío estándar del RTT: No se puede calcular con un solo valor")
    else:
        print("No se puede calcular RTTs: no hubo respuestas válidas")


###INPUT DE PAQUETES
host_input = input("Ingresá el host al que querés hacer ping (ej: google.com): ")
cantidad_input = int(input("¿Cuántos paquetes querés enviar?: "))

ping_estadisticas(host_input, cantidad_input)

Ingresá el host al que querés hacer ping (ej: google.com): www.google.com
¿Cuántos paquetes querés enviar?: 200
Enviando 200 paquetes ICMP a www.google.com...

[#1] Respuesta de 142.251.188.147 | Longitud: 28 bytes | RTT: 26.20 ms | TTL: 114
[#2] Respuesta de 142.251.188.147 | Longitud: 28 bytes | RTT: 23.61 ms | TTL: 114
[#3] Respuesta de 142.251.188.147 | Longitud: 28 bytes | RTT: 20.48 ms | TTL: 114
[#4] Respuesta de 142.251.188.147 | Longitud: 28 bytes | RTT: 22.51 ms | TTL: 114
[#5] Respuesta de 142.251.188.147 | Longitud: 28 bytes | RTT: 32.58 ms | TTL: 114
[#6] Respuesta de 142.251.188.147 | Longitud: 28 bytes | RTT: 20.63 ms | TTL: 114
[#7] Respuesta de 142.251.188.147 | Longitud: 28 bytes | RTT: 20.61 ms | TTL: 114
[#8] Respuesta de 142.251.188.147 | Longitud: 28 bytes | RTT: 21.00 ms | TTL: 114
[#9] Respuesta de 142.251.188.147 | Longitud: 28 bytes | RTT: 20.53 ms | TTL: 114
[#10] Respuesta de 142.251.188.147 | Longitud: 28 bytes | RTT: 20.45 ms | TTL: 114
[#11] Respuesta de 

Ej 3. Se pide listar todos los tipos de errores de ICMP de tipo 3


> Tipo 3 – Destination Unreachable (Destino inalcanzable):

Código 0: Network unreachable

Código 1: Host unreachable

Código 2: Protocol unreachable

Código 3: Port unreachable

Código 4: Fragmentation needed and DF set

Código 5: Source route failed

Código 6: Destination network unknown

Código 7: Destination host unknown

Código 8: Source host isolated

Código 9: Network administratively prohibited

Código 10: Host administratively prohibited

Código 11: Network unreachable for TOS

Código 12: Host unreachable for TOS

Código 13: Communication administratively prohibited

Código 14: Host precedence violation

Código 15: Precedence cutoff in effect


Fuente: RFC 792 -> https://datatracker.ietf.org/doc/html/rfc792


In [10]:
from scapy.all import *
from scapy.layers.inet import IP, ICMP
import time
import statistics

#errores icmp de tipo 3
ICMP_DEST_UNREACH_CODES = {
    0: "Network unreachable",
    1: "Host unreachable",
    2: "Protocol unreachable",
    3: "Port unreachable",
    4: "Fragmentation needed and DF set",
    5: "Source route failed",
    6: "Destination network unknown",
    7: "Destination host unknown",
    8: "Source host isolated",
    9: "Network administratively prohibited",
    10: "Host administratively prohibited",
    11: "Network unreachable for TOS",
    12: "Host unreachable for TOS",
    13: "Communication administratively prohibited",
    14: "Host precedence violation",
    15: "Precedence cutoff in effect"
}

def ping_estadisticas(host, cantidad):
    print(f"Enviando {cantidad} paquetes ICMP a {host}...\n")

    enviados = 0
    recibidos = 0
    rtts = []

    # Resolución DNS usando Scapy
    try:
        ip_destino = conf.resolve(host)
    except Exception as e:
        print(f"Error de DNS: “Name not known”")
        return

    for i in range(cantidad):
        pkt = IP(dst=ip_destino)/ICMP(id=0x1234, seq=i)
        enviados += 1

        start_time = time.time()
        resp = sr1(pkt, timeout=2, verbose=0)
        end_time = time.time()

        if resp is not None:
            if ICMP in resp:
                icmp_layer = resp[ICMP]
                if icmp_layer.type == 0:
                    rtt = (end_time - start_time) * 1000
                    rtts.append(rtt)
                    recibidos += 1
                    print(f"[#{i+1}] Respuesta de {resp.src} | Longitud: {len(resp)} bytes | RTT: {rtt:.2f} ms | TTL: {resp.ttl}")
                elif icmp_layer.type == 3:
                    code = icmp_layer.code
                    descripcion = ICMP_DEST_UNREACH_CODES.get(code, "Código desconocido")
                    print(f"[#{i+1}] ICMP Error: Destination Unreachable - {descripcion} (Código {code}) desde {resp.src}")
                else:
                    print(f"[#{i+1}] Recibido ICMP tipo {icmp_layer.type}, no es respuesta ni error manejado.")
            else:
                print(f"[#{i+1}] Paquete recibido sin capa ICMP")
        else:
            print(f"[#{i+1}] No se recibió respuesta")

    # Estadísticas
    perdidos = enviados - recibidos
    porcentaje_perdidos = (perdidos / enviados) * 100 if enviados > 0 else 0

    print("\nEstadísticas finales:")
    print(f"Paquetes enviados: {enviados}")
    print(f"Paquetes recibidos: {recibidos}")
    print(f"Paquetes perdidos: {perdidos}")
    print(f"Porcentaje de pérdida: {porcentaje_perdidos:.2f}%")

    if rtts:
        print(f"RTT promedio: {statistics.mean(rtts):.2f} ms")
        print(f"RTT máximo: {max(rtts):.2f} ms")
        print(f"RTT mínimo: {min(rtts):.2f} ms")
        if len(rtts) > 1:
            print(f"Desvío estándar del RTT: {statistics.stdev(rtts):.2f} ms")
        else:
            print("Desvío estándar del RTT: No se puede calcular con un solo valor")
    else:
        print("No se puede calcular RTTs: no hubo respuestas válidas")

# Entradas
host_input = input("Ingresá el host al que querés hacer ping (ej: google.com): ")
cantidad_input = int(input("¿Cuántos paquetes querés enviar?: "))

ping_estadisticas(host_input, cantidad_input)


Ingresá el host al que querés hacer ping (ej: google.com): www.google.com
¿Cuántos paquetes querés enviar?: 100
Enviando 100 paquetes ICMP a www.google.com...

Error de DNS: “Name not known”


EJ4. Se pide medir los tiempos de RTT en 5 host de 5 universidades de continentes distintos

In [15]:
import subprocess
import re
import pandas as pd

# Lista de universidades por continente
universidades = {
    "América del Norte": "harvard.edu",              # USA
    "América del Sur": "uba.ar",                     # Argentina (UBA)
    "Europa": "ox.ac.uk",                            # Reino Unido (Oxford)
    "Asia": "tsinghua.edu.cn",                       # China (Tsinghua)
    "Oceanía": "unimelb.edu.au",                     # Australia (Melbourne)

}

def hacer_ping(host, count=4):
    try:
        output = subprocess.check_output(["ping", "-c", str(count), host], stderr=subprocess.STDOUT, universal_newlines=True)
        match = re.search(r"rtt.* = ([\d\.]+)/([\d\.]+)/([\d\.]+)", output)
        if match:
            min_rtt, avg_rtt, max_rtt = map(float, match.groups())
            return avg_rtt
        else:
            return None
    except subprocess.CalledProcessError as e:
        print(f"Error con {host}:\n{e.output}")
        return None

# Ejecutar ping y guardar resultados
resultados = []

for continente, dominio in universidades.items():
    print(f"Haciendo ping a {dominio} ({continente})...")
    rtt = hacer_ping(dominio)
    resultados.append({
        "Continente": continente,
        "Universidad": dominio,
        "RTT promedio (ms)": round(rtt, 2) if rtt else "Error"
    })

# Mostrar resultados en tabla
df = pd.DataFrame(resultados)
import seaborn as sns
import matplotlib.pyplot as plt
import numpy as np

display(df)

# Gráfico opcional
df_valores = df[df["RTT promedio (ms)"] != "Error"]
df_valores["RTT promedio (ms)"] = df_valores["RTT promedio (ms)"].astype(float)

sns.barplot(x="Continente", y="RTT promedio (ms)", data=df_valores)
plt.title("RTT promedio por universidad (ping)")
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


Haciendo ping a harvard.edu (América del Norte)...


FileNotFoundError: [Errno 2] No such file or directory: 'ping'

A este punto lo automatizo yo desde N8N

Realizar mediciones en diferentes momentos del día (mañana, tarde, noche) hacia la misma universidad durante una semana. ¿Hay diferencias en el RTT para diferentes momentos del día? Si las hay, ¿A qué puede deberse?